We will first load the data and scale it

In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

df_train = pd.read_csv('./dataset/train.csv')
df_val = pd.read_csv('./dataset/val.csv')
df_test = pd.read_csv('./dataset/test.csv')

scaler_r = MinMaxScaler()
scaler_c = MinMaxScaler()

scaler_r.fit(df_test[['registered']])
scaler_c.fit(df_test[['casual']])


df_train['registered_s'] = scaler_r.transform(df_train[['registered']])
df_val['registered_s'] = scaler_r.transform(df_val[['registered']])

df_train['casual_s'] = scaler_c.transform(df_train[['casual']])
df_val['casual_s'] = scaler_c.transform(df_val[['casual']])

In [2]:
df_train.head()

,Unnamed: 0,season,workingday,weathersit,temp,hum,casual,registered,day_sin,day_cos,week_sin,week_cos,registered_s,casual_s
0,0,1,0,1,0.24,0.81,3,13,0.000000,1.000000,-0.781831,0.62349,0.014840,0.008646
1,1,1,0,1,0.22,0.80,8,32,0.258819,0.965926,-0.781831,0.62349,0.036530,0.023055
2,2,1,0,1,0.22,0.80,5,27,0.500000,0.866025,-0.781831,0.62349,0.030822,0.014409
3,3,1,0,1,0.24,0.75,3,10,0.707107,0.707107,-0.781831,0.62349,0.011416,0.008646
4,4,1,0,1,0.24,0.75,0,1,0.866025,0.500000,-0.781831,0.62349,0.001142,0.000000


In [3]:
df_train.drop(['Unnamed: 0','casual','registered'],axis=1,inplace=True)
df_val.drop(['Unnamed: 0','casual','registered'],axis=1,inplace=True)

df_train.head()

,season,workingday,weathersit,temp,hum,day_sin,day_cos,week_sin,week_cos,registered_s,casual_s
0,1,0,1,0.24,0.81,0.000000,1.000000,-0.781831,0.62349,0.014840,0.008646
1,1,0,1,0.22,0.80,0.258819,0.965926,-0.781831,0.62349,0.036530,0.023055
2,1,0,1,0.22,0.80,0.500000,0.866025,-0.781831,0.62349,0.030822,0.014409
3,1,0,1,0.24,0.75,0.707107,0.707107,-0.781831,0.62349,0.011416,0.008646
4,1,0,1,0.24,0.75,0.866025,0.500000,-0.781831,0.62349,0.001142,0.000000


Now Lets create the dataset from the tabular data to work with our model

In [4]:
import numpy as np


def create_dataset(data, feature_columns, target_columns, lookback_window, horizon, shift):
    """
    Transforms a time series DataFrame into input-target pairs for a deep learning model.

    Args:
        data (pd.DataFrame): The input DataFrame.
        feature_columns (list): List of column names to use as input features (X).
        target_columns (list): List of column names to use as target variables (Y).
        lookback_window (int): The number of past time steps to use as input (X).
        horizon (int): The number of future time steps to predict (Y).
        shift (int): The number of steps to shift the window for each new sample.

    Returns:
        tuple: A tuple containing two NumPy arrays, X (inputs) and Y (targets).
    """
    X, Y = [], []
    for i in range(0, len(data) - lookback_window - horizon + 1, shift):
        # Extract the input features (X) for the current window
        x_start = i
        x_end = i + lookback_window
        X.append(data.iloc[x_start:x_end][feature_columns].values)

        # Extract the target variables (Y) for the forecast horizon
        y_start = i + lookback_window
        y_end = y_start + horizon
        Y.append(data.iloc[y_start:y_end][target_columns].values)
    
    return np.array(X), np.array(Y)


In [ ]:
import yaml

def load_config(filepath):
    """
    Loads a YAML configuration file from the specified filepath.

    Args:
        filepath (str): The path to the YAML file.

    Returns:
        dict: The configuration as a dictionary.
    """
    try:
        with open(filepath, 'r') as file:
            config = yaml.safe_load(file)
        return config
    except FileNotFoundError:
        print(f"Error: The file at {filepath} was not found.")
        return None
    except yaml.YAMLError as exc:
        print(f"Error parsing YAML file: {exc}")
        return None

# Load the configuration from the YAML file
config_file = 'config.yaml'
model_config = load_config(config_file)

In [ ]:
#  Define key parameters
LOOKBACK_WINDOW = model_config['dataset']['lookback_window']  
FORECAST_HORIZON = model_config['dataset']['forecast_horizon'] 
SHIFT = model_config['dataset']['shift']          
# ----------------------------------------------------
# The columns model use to learn the patterns.

feature_columns = model_config['data']['feature_columns']

# The columns model will try to predict.
target_columns = model_config['data']['target_columns']



Lets' first create the training dataset and have a look at it

In [6]:
X_train, Y_train = create_dataset(df_train, feature_columns, target_columns, LOOKBACK_WINDOW, FORECAST_HORIZON, SHIFT)
X_val, Y_val = create_dataset(df_val, feature_columns, target_columns, LOOKBACK_WINDOW, FORECAST_HORIZON, SHIFT)


print(f"Shape of Input (X): {X_train.shape}")       # shape will be in the following order
print(f"Shape of Target (Y): {Y_train.shape}")      # [Number of Samples, Window, Number of Features]


Shape of Input (X): (13877, 24, 7)
Shape of Target (Y): (13877, 3, 2)


Now Let's create dataloader which will serve the above created data to our models for training and testing.

In [7]:
import torch
from torch.utils.data import DataLoader, TensorDataset


# Convert NumPy arrays to PyTorch Tensors
X_train = torch.Tensor(X_train)
y_train = torch.Tensor(Y_train)
X_val = torch.Tensor(X_val)
y_val = torch.Tensor(Y_val)

# Create TensorDatasets and DataLoaders
train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)


Now let's Design our model.  
We will create two models. one solely based on LSTM architecture and another one combining CNN with LSTM.
Design is done in the model.py script so that it can be used across the notebooks


In [8]:
from model import LSTMForecaster,CNN_LSTMForecaster

Let's initialize the both models.

In [ ]:
import torch.nn as nn

BATCH_SIZE = model_config['model']['batch_size']
LOOKBACK_WINDOW = model_config['dataset']['lookback_window']  
NUM_FEATURES = model_config['model']['num_features']
FORECAST_HORIZON = model_config['dataset']['forecast_horizon'] 
NUM_TARGETS = model_config['model']['num_targets']


device = 'cuda' if torch.cuda.is_available() else 'cpu'

lstm_model = LSTMForecaster(
        num_features=NUM_FEATURES,
        hidden_size=64,
        num_layers=1,
        output_size=FORECAST_HORIZON * NUM_TARGETS
    )

cnn_lstm_model = CNN_LSTMForecaster(
        num_features=NUM_FEATURES,
        hidden_size=50,
        num_layers=2,
        output_size=FORECAST_HORIZON * NUM_TARGETS
    )

In [10]:
from tqdm import tqdm

def train_model(model, optimizer, loss_fn, train_loader, val_loader, epochs=20, device='cpu', name=''):
    """
    Trains and validates a time series forecasting model.

    Args:
        model (nn.Module): The forecasting model to train.
        optimizer (torch.optim.Optimizer): The optimizer to use for training.
        loss_fn (nn.Module): The loss function (e.g., MSELoss).
        train_loader (DataLoader): DataLoader for the training data.
        val_loader (DataLoader): DataLoader for the validation data.
        epochs (int): The number of training epochs.
        device (str): The device to run the training on ('cpu' or 'cuda').
    """
    best_val_loss = float('inf')
    best_model_state = None

    model.to(device)

    for epoch in range(epochs):
        # --- Training Loop ---
        model.train()
        train_loss = 0.0
        
        for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs} Training"):
            optimizer.zero_grad() # Reset gradients
            
            inputs,targets = batch
            inputs=inputs.to(device)
            targets=targets.to(device)

            output = model(inputs)
            
            loss = loss_fn(output, targets)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item() * targets.size(0)
        
        avg_train_loss = train_loss / len(train_loader.dataset)

        # --- Validation Loop ---
        model.eval()
        val_loss = 0.0
        
        with torch.no_grad():
            for batch in tqdm(val_loader, desc=f"Epoch {epoch+1}/{epochs} Validating"):
                optimizer.zero_grad() # Reset gradients
                
                inputs,targets = batch
                inputs=inputs.to(device)
                targets=targets.to(device)

                output = model(inputs)
                
                loss = loss_fn(output, targets)
                
                val_loss += loss.item() * targets.size(0)

        # Calculate average validation loss for the epoch
        avg_val_loss = val_loss / len(val_loader.dataset)
        
        print(f"Epoch {epoch+1}/{epochs}: Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")

        # Check if current validation loss is the best so far
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            # Save the model state dictionary
            best_model_state = model.state_dict()
            print("Validation loss improved. Saving best model state.")
    
    if best_model_state:
        torch.save(best_model_state, './models/'+name+'_best.pth') # Save the best model
        print("Training complete. models saved to models/")
    else:
        print("Training complete. Could not save best model state.")
    

In [11]:
model = lstm_model
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nn.L1Loss()

print("\nStarting model training...")
train_model(
    model=model,
    optimizer=optimizer,
    loss_fn=loss_fn,
    train_loader=train_loader,
    val_loader=val_loader,
    epochs=25, 
    device=device,
    name='lstm_L1'
)

lstm_model=model


Starting model training...


Epoch 1/25 Validating: 100%|██████████| 27/27 [00:00<00:00, 744.50it/s]


Epoch 1/25: Train Loss: 0.0609, Val Loss: 0.0883
Validation loss improved. Saving best model state.


Epoch 2/25 Validating: 100%|██████████| 27/27 [00:00<00:00, 628.71it/s]


Epoch 2/25: Train Loss: 0.0636, Val Loss: 0.0844
Validation loss improved. Saving best model state.


...

Epoch 23/25: Train Loss: 0.0426, Val Loss: 0.0500


Epoch 24/25 Validating: 100%|██████████| 27/27 [00:00<00:00, 598.82it/s]


Epoch 24/25: Train Loss: 0.0423, Val Loss: 0.0501


Epoch 25/25 Validating: 100%|██████████| 27/27 [00:00<00:00, 639.14it/s]

Epoch 25/25: Train Loss: 0.0419, Val Loss: 0.0498
Validation loss improved. Saving best model state.
Training complete. models saved to models/


In [12]:
model = cnn_lstm_model
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nn.L1Loss()

print("\nStarting model training...")
train_model(
    model=model,
    optimizer=optimizer,
    loss_fn=loss_fn,
    train_loader=train_loader,
    val_loader=val_loader,
    epochs=25,
    device=device,
    name='cnn_lstm_L1'
)

cnn_lstm_model=model


Starting model training...


Epoch 1/25 Validating: 100%|██████████| 27/27 [00:00<00:00, 487.71it/s]


Epoch 1/25: Train Loss: 0.0627, Val Loss: 0.0887
Validation loss improved. Saving best model state.


Epoch 2/25 Validating: 100%|██████████| 27/27 [00:00<00:00, 321.74it/s]


Epoch 2/25: Train Loss: 0.0633, Val Loss: 0.0786
Validation loss improved. Saving best model state.


...

Epoch 23/25: Train Loss: 0.0402, Val Loss: 0.0480
Validation loss improved. Saving best model state.


Epoch 24/25 Validating: 100%|██████████| 27/27 [00:00<00:00, 486.46it/s]


Epoch 24/25: Train Loss: 0.0399, Val Loss: 0.0479
Validation loss improved. Saving best model state.


Epoch 25/25 Validating: 100%|██████████| 27/27 [00:00<00:00, 534.56it/s]

Epoch 25/25: Train Loss: 0.0396, Val Loss: 0.0477
Validation loss improved. Saving best model state.
Training complete. models saved to models/
